# Translating French to English with Pytorch

**
[Ted Note] Please MUST practice this Pytorch tutorial first to gain clean understanding for simple version
**

**
[Translation with a Sequence to Sequence Network and Attention][1]
**

[1]: http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [1]:
%matplotlib inline
import re, pickle, collections, bcolz, numpy as np, keras, sklearn, math, operator 
from numpy import random

C:\ProgramData\Anaconda3\envs\pytorchenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#import gensim
from gensim.models import word2vec

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F

C:\ProgramData\Anaconda3\envs\pytorchenv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
#path='/data/datasets/fr-en-109-corpus/'
#dpath = 'data/translate/'
path = '../../../Data/fr-en-109-corpus/'
dpath = '../../../Data/translate/'
glove_path = '../../../Data/'

## Prepare corpus

The French-English parallel corpus can be downloaded from http://www.statmt.org/wmt10/training-giga-fren.tar. It was created by Chris Callison-Burch, who crawled millions of web pages and then used 'a set of simple heuristics to transform French URLs onto English URLs (i.e. replacing "fr" with "en" and about 40 other hand-written rules), and assume that these documents are translations of each other'.

In [4]:
fname=path+'giga-fren.release2.fixed'
en_fname = fname+'.en'
fr_fname = fname+'.fr'

To make this problem a little simpler so we can train our model more quickly, we'll just learn to translate questions that begin with 'Wh' (e.g. what, why, where which). Here are our regexps that filter the sentences we want.

In [5]:
re_eq = re.compile('^(Wh[^?.!]+\?)')
re_fq = re.compile('^([^?.!]+\?)')

lines = ((re_eq.search(eq), re_fq.search(fq)) 
         for eq, fq in zip(open(en_fname, encoding='utf8'), open(fr_fname, encoding='utf8')))

qs = [(e.group(), f.group()) for e, f in lines if e and f]
print(len(qs))

52331


In [6]:
qs[:6]

[('What is light ?', 'Qu’est-ce que la lumière?'),
 ('Who are we?', 'Où sommes-nous?'),
 ('Where did we come from?', "D'où venons-nous?"),
 ('What would we do without it?', 'Que ferions-nous sans elle ?'),
 ('What is the absolute location (latitude and longitude) of Badger, Newfoundland and Labrador?',
  'Quelle sont les coordonnées (latitude et longitude) de Badger, à Terre-Neuve-etLabrador?'),
 ('What is the major aboriginal group on Vancouver Island?',
  'Quel est le groupe autochtone principal sur l’île de Vancouver?')]

Because it takes a while to load the data, we save the results to make it easier to load in later.

In [7]:
pickle.dump(qs, open(path+'fr-en-qs.pkl', 'wb'))

In [8]:
qs = pickle.load(open(path+'fr-en-qs.pkl', 'rb'))

In [9]:
en_qs, fr_qs = zip(*qs)

Because we are translating at word level, we need to tokenize the text first. (Note that it is also possible to translate at character level, which doesn't require tokenizing.) There are many tokenizers available, but we found we got best results using these simple heuristics.

In [10]:
re_apos = re.compile(r"(\w)'s\b")         # make 's a separate word
re_mw_punc = re.compile(r"(\w[’'])(\w)")  # other ' in a word creates 2 words
re_punc = re.compile("([\"().,;:/_?!—])") # add spaces around punctuation
re_mult_space = re.compile(r"  *")        # replace multiple spaces with just one

def simple_toks(sent):
    sent = re_apos.sub(r"\1 's", sent)
    sent = re_mw_punc.sub(r"\1 \2", sent)
    sent = re_punc.sub(r" \1 ", sent).replace('-', ' ')
    sent = re_mult_space.sub(' ', sent)
    return sent.lower().split()

In [11]:
fr_qtoks = list(map(simple_toks, fr_qs)); fr_qtoks[:4]

[['qu’', 'est', 'ce', 'que', 'la', 'lumière', '?'],
 ['où', 'sommes', 'nous', '?'],
 ["d'", 'où', 'venons', 'nous', '?'],
 ['que', 'ferions', 'nous', 'sans', 'elle', '?']]

In [12]:
en_qtoks = list(map(simple_toks, en_qs)); en_qtoks[:4]

[['what', 'is', 'light', '?'],
 ['who', 'are', 'we', '?'],
 ['where', 'did', 'we', 'come', 'from', '?'],
 ['what', 'would', 'we', 'do', 'without', 'it', '?']]

In [13]:
simple_toks("Rachel's baby is cuter than other's.")

['rachel', "'s", 'baby', 'is', 'cuter', 'than', 'other', "'s", '.']

Special tokens used to pad the end of sentences, and to mark the start of a sentence.

In [14]:
PAD = 0; SOS = 1

Enumerate the unique words (*vocab*) in the corpus, and also create the reverse map (word->index). Then use this mapping to encode every sentence as a list of int indices.

In [15]:
def toks2ids(sents):
    voc_cnt = collections.Counter(t for sent in sents for t in sent)
    vocab = sorted(voc_cnt, key=voc_cnt.get, reverse=True)
    vocab.insert(PAD, "<PAD>")
    vocab.insert(SOS, "<SOS>")
    w2id = {w:i for i,w in enumerate(vocab)}
    ids = [[w2id[t] for t in sent] for sent in sents]
    return ids, vocab, w2id, voc_cnt

In [16]:
fr_ids, fr_vocab, fr_w2id, fr_counts = toks2ids(fr_qtoks)
en_ids, en_vocab, en_w2id, en_counts = toks2ids(en_qtoks)

In [17]:
[[en_vocab[wid] for wid in s] for s in en_ids[:3]]

[['what', 'is', 'light', '?'],
 ['who', 'are', 'we', '?'],
 ['where', 'did', 'we', 'come', 'from', '?']]

## Word vectors

Stanford's GloVe word vectors can be downloaded from https://nlp.stanford.edu/projects/glove/ (in the code below we have preprocessed them into a bcolz array). We use these because each individual word has a single word vector, which is what we need for translation. Word2vec, on the other hand, often uses multi-word phrases.

In [18]:
def load_glove(loc):
    return (bcolz.open(loc+'.dat')[:],
        pickle.load(open(loc+'_words.pkl','rb'), encoding='latin1'),
        pickle.load(open(loc+'_wordidx.pkl','rb'), encoding='latin1'))

In [19]:
en_vecs, en_wv_word, en_wv_idx = load_glove(glove_path+'/glove.6B/results/100d')
en_w2v = {w: en_vecs[en_wv_idx[w]] for w in en_wv_word}
n_en_vec, dim_en_vec = en_vecs.shape

In [20]:
en_w2v['king']

array([-0.32307 , -0.87616 ,  0.21977 ,  0.25268 ,  0.22976 ,  0.7388  ,
       -0.37954 , -0.35307 , -0.84369 , -1.1113  , -0.30266 ,  0.33178 ,
       -0.25113 ,  0.30448 , -0.077491, -0.89815 ,  0.092496, -1.1407  ,
       -0.58324 ,  0.66869 , -0.23122 , -0.95855 ,  0.28262 , -0.078848,
        0.75315 ,  0.26584 ,  0.3422  , -0.33949 ,  0.95608 ,  0.065641,
        0.45747 ,  0.39835 ,  0.57965 ,  0.39267 , -0.21851 ,  0.58795 ,
       -0.55999 ,  0.63368 , -0.043983, -0.68731 , -0.37841 ,  0.38026 ,
        0.61641 , -0.88269 , -0.12346 , -0.37928 , -0.38318 ,  0.23868 ,
        0.6685  , -0.43321 , -0.11065 ,  0.081723,  1.1569  ,  0.78958 ,
       -0.21223 , -2.3211  , -0.67806 ,  0.44561 ,  0.65707 ,  0.1045  ,
        0.46217 ,  0.19912 ,  0.25802 ,  0.057194,  0.53443 , -0.43133 ,
       -0.34311 ,  0.59789 , -0.58417 ,  0.068995,  0.23944 , -0.85181 ,
        0.30379 , -0.34177 , -0.25746 , -0.031101, -0.16285 ,  0.45169 ,
       -0.91627 ,  0.64521 ,  0.73281 , -0.22752 , 

For French word vectors, we're using those from http://fauconnier.github.io/index.html

In [21]:
import word2vec
fr_model = word2vec.load(glove_path+'frWac_non_lem_no_postag_no_phrase_200_skip_cut100.bin')
fr_voc = fr_model.vocab
dim_fr_vec = 200

We need to map each word index in our vocabs to their word vector. Not every word in our vocabs will be in our word vectors, since our tokenization approach won't be identical to the word vector creators - in these cases we simply create a random vector.

In [22]:
def create_emb(w2v, targ_vocab, dim_vec):
    vocab_size = len(targ_vocab)
    emb = np.zeros((vocab_size, dim_vec))
    found = 0
    
    for i, word in enumerate(targ_vocab):
        try: emb[i] = w2v[word]; found+=1
        except KeyError: emb[i] = np.random.normal(scale=0.6, size=(dim_vec,))
            
    return emb, found

In [23]:
en_embs, found = create_emb(en_w2v, en_vocab, dim_en_vec); en_embs.shape, found

((19549, 100), 17251)

In [24]:
fr_embs, found = create_emb(fr_model, fr_vocab, dim_fr_vec); fr_embs.shape, found

((26709, 200), 21878)

## Prep data

Each sentence has to be of equal length. Keras has a convenient function `pad_sequences` to truncate and/or pad each sentence as required - even although we're not using keras for the neural net, we can still use any functions from it we need!

In [25]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 30
en_padded = pad_sequences(en_ids, maxlen, 'int64', 'post', 'post')
fr_padded = pad_sequences(fr_ids, maxlen, 'int64', 'post', 'post')
en_padded.shape, fr_padded.shape, en_embs.shape, fr_embs.shape

((52331, 30), (52331, 30), (19549, 100), (26709, 200))

And of course we need to separate our training and test sets...

In [26]:
from sklearn.model_selection import train_test_split
fr_train, fr_test, en_train, en_test = train_test_split(en_padded, fr_padded, test_size=0.1)

print([o.shape for o in (fr_train, fr_test, en_train, en_test)])

[(47097, 30), (5234, 30), (47097, 30), (5234, 30)]


Here's an example of a French and English sentence, after encoding and padding.

In [27]:
fr_train[0], en_train[0]

(array([  29, 1418, 5898,   10,    3,   66,   89,   13,   17,    2,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0], dtype=int64),
 array([  19,   13,    4, 1370, 7990,   17, 5395,    6,   45,   30,  365,
          14,   27,    2,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0], dtype=int64))

## Model

### Basic encoder-decoder

In [28]:
def long_t(arr): return Variable(torch.LongTensor(arr)).cuda()

In [29]:
fr_emb_t = torch.FloatTensor(fr_embs).cuda()
en_emb_t = torch.FloatTensor(en_embs).cuda()

In [30]:
def create_emb(emb_mat, non_trainable=False):
    output_size, emb_dim = emb_mat.size() # emb_mat.shape also works
    emb = nn.Embedding(output_size, emb_dim)
    emb.load_state_dict({'weight': emb_mat})
    if non_trainable:
        for param in emb.parameters():
            param.requires_grad = False
    return emb, emb_dim, output_size

Turning a sequence into a representation can be done using an RNN (called the 'encoder'. This approach is useful because RNN's are able to keep track of state and memory, which is obviously important in forming a complete understanding of a sentence.
* `bidirectional=True` passes the original sequence through an RNN, and the reversed sequence through a different RNN and concatenates the results. This allows us to look forward and backwards.
* We do this because in language things that happen later often influence what came before (i.e. in Spanish, "el chico, la chica" means the boy, the girl; the word for "the" is determined by the gender of the subject, which comes after).

In [31]:
class EncoderRNN(nn.Module):
    
    def __init__(self, embs, hidden_size, n_layers=2, n_directions=1):
        '''
            Args:
                embs: in shape (input_size, emb_dim). 
                      ps. output_size = input_size from encoder perspective
        '''
        super(EncoderRNN, self).__init__()
        self.emb, emb_size, output_size = create_emb(embs, True)
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.n_directions = n_directions
        if n_directions == 1:
            self.gru = nn.GRU(emb_size, hidden_size, num_layers=n_layers, 
                          batch_first=True)
        elif n_directions == 2:
            self.gru = nn.GRU(emb_size, hidden_size, num_layers=n_layers, 
                          batch_first=True, bidirectional=True)
    
    def forward(self, input, hidden):
        '''
            Args:
                - inp: Input of nn.Embedding() 
                    LongTensor (N, W), 
                    N = mini-batch, 
                    W = number of indices to extract per mini-batch
            Outputs: Outputs of nn.GRU() 
                - output: (batch, seq_len, hidden_size * num_directions)
                - h_n: (num_layers * num_directions, batch, hidden_size)
        '''
        return self.gru(self.emb(input), hidden)
    
    def initHidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers*self.n_directions, 
                                    batch_size, 
                                    self.hidden_size)).cuda()

In [32]:
def encode(inp, encoder):
    batch_size, input_length = inp.size()
    hidden = encoder.initHidden(batch_size)
    enc_outputs, hidden= encoder(inp, hidden)
    return long_t([SOS]*batch_size), enc_outputs, hidden

Finally, we arrive at a vector representation of the sequence which captures everything we need to translate it. We feed this vector into more RNN's, which are trying to generate the labels. After this, we make a classification for what each word is in the output sequence.

In [33]:
class DecoderRNN(nn.Module):
    def __init__(self, embs, hidden_size, n_layers=2):
        '''
            Args:
                embs: in shape (input_size, emb_dim). 
                      ps. output_size = input_size from encoder perspective
        '''
        super(DecoderRNN, self).__init__()
        self.emb, emb_size, output_size = create_emb(embs, False)
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(emb_size, hidden_size, num_layers=n_layers, 
                          batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, inp, hidden):
        '''
            Args:
                - inp: Input of nn.Embedding()- 
                    LongTensor (N, W), 
                    N = mini-batch, 
                    W = number of indices to extract per mini-batch
        '''
        # emb output: (batch_size, emb_dim)
        # unsqueeze(1) to turn into (batch_size, 1, emb_dim)
        emb = self.emb(inp).unsqueeze(1)
        output = emb #F.relu(embedded)
        output, hidden = self.gru(output, hidden)
        # feed self.out Linear layer by GRU output 
        # shape (batch_size, seq_len, hidden_size) 
        # into shape (batch_size, hidden_size)
        res = F.log_softmax(self.out(output[:,0]), dim=1)
        return res, hidden
    
    def initHidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, 
                                    batch_size, 
                                    self.hidden_size)).cuda()

This graph demonstrates the accuracy decay for a neural translation task. With an encoding/decoding technique, larger input sequences result in less accuracy.

<img src="https://smerity.com/media/images/articles/2016/bahdanau_attn.png" width="600">

This can be mitigated using an attentional model.

### Attentional model

In [34]:
def Arr(*sz): return torch.randn(sz)/math.sqrt(sz[0])

In [35]:
def Var(*sz): return Parameter(Arr(*sz)).cuda()

In [36]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, embs, hidden_size, n_layers=2, 
                 dropout_p=0.1, max_length=maxlen,
                 n_encoder_directions=1):
        super(AttnDecoderRNN, self).__init__()
        self.emb, emb_size, output_size = create_emb(embs, False)
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.n_encoder_directions= n_encoder_directions
        
        # ** unique for AttnDecoder **
        self.attn = nn.Linear(emb_size+self.hidden_size, self.max_length)
        self.attn_combine = nn.Linear(emb_size+self.hidden_size*self.n_encoder_directions, 
                                      hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=n_layers, 
                          batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, inp, hidden, encoder_outputs):
        '''
            Args:
                - inp: Input of nn.Embedding()- 
                    LongTensor (N, W), 
                    N = mini-batch, 
                    W = number of indices to extract per mini-batch
                ** unique for AttnDecoder ** 
                - encoder_outputs: (batch_size, self.max_length, hidden_size*n_encoder_directions)
                   a -- hidden state output of the Bi-GRU/Bi-LSTM encoder
        '''
        # emb output: (batch_size, emb_dim)
        # [unsqueeze(1) to turn into (batch_size, 1, emb_dim)]
        emb = self.emb(inp) #.unsqueeze(1)
        emb = self.dropout(emb)
        
        # concat in input embedded a and hidden s(t-1) 
        # in shape (batch_size, emb_dim) and (batch_size, hidden_size)
        # turn into shape (batch_size, self.max_length) by self.attn Dense Layer
        attn_weights = F.softmax(
            self.attn(torch.cat((emb, hidden[0]), 1)), dim=1)
        
        # torch.bmm input and output shapes: 
        # Inputs:
        #     - attn_weights: (batch_size, self.max_length)
        #     - encoder_outputs: (batch_size, self.max_length, hidden_size*n_encoder_directions)
        # Output: 
        #     - attn_applied: (batch_size, seq_len=1, hidden_size*n_encoder_directions)
        #        
        attn_applied = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs) 
        
        # output shape: (batch_size, emb_dim+hidden_size*n_encoder_directions) 
        output = torch.cat((emb, attn_applied[:,0]), 1)
        
        # output shape: (batch_size, hidden_size)
        # unsqueeze(1) to turn into (batch_size, 1, hidden_size)
        output = self.attn_combine(output).unsqueeze(1)

        
        output, hidden = self.gru(output, hidden)
        
        # feed self.out Linear layer by GRU output 
        # shape (batch_size, seq_len, hidden_size) 
        # into shape (batch_size, hidden_size)
        output = F.log_softmax(self.out(output[:,0]), dim=1)
        return output, hidden, attn_weights
    
    def initHidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, 
                                    batch_size, 
                                    self.hidden_size)).cuda()

### Encoder, Decoder, Attention testing

Pytorch makes it easy to check intermediate results, when creating a custom architecture such as this one, since you can interactively run each function.

In [37]:
def get_batch(x, y, batch_size=16):
    idxs = np.random.permutation(len(x))[:batch_size]
    return x[idxs], y[idxs]

In [38]:
hidden_size = 128
batch_size = 4
fra, eng = get_batch(fr_train, en_train, batch_size)
inp = long_t(fra)
targ = long_t(eng)

emb, emb_size, output_size = create_emb(fr_emb_t, True)
emb.cuda()
print(inp.size()) # shape: (batch_size, seq_len)
print(emb(inp).shape) # shape: (batch_size, seq_len, emb_dim)

torch.Size([4, 30])
torch.Size([4, 30, 200])


In [39]:
encoder = EncoderRNN(fr_emb_t, hidden_size).cuda()

In [40]:
hidden = encoder.initHidden(batch_size).cuda()
gru = nn.GRU(emb_size, hidden_size, num_layers=2, 
                          batch_first=True).cuda()
enc_outputs, hidden = gru(emb(inp), hidden)

In [41]:
dec_input, enc_outputs, hidden = encode(inp, encoder)
print(dec_input.shape) # (batch_size,)
print(enc_outputs.shape) # (batch_size, seq_len, hidden_size)
print(hidden.shape) # (n_layers, batch_size, hidden_size)

torch.Size([4])
torch.Size([4, 30, 128])
torch.Size([2, 4, 128])


In [42]:
encoder.n_directions

1

In [43]:
decoder = DecoderRNN(en_emb_t, hidden_size).cuda()

In [44]:
hidden = decoder.initHidden(batch_size).cuda()

In [45]:
dec_outputs, hidden = decoder(dec_input, hidden)

In [46]:
print(dec_input.shape); print(dec_input)

torch.Size([4])
Variable containing:
 1
 1
 1
 1
[torch.cuda.LongTensor of size 4 (GPU 0)]



In [47]:
print(dec_outputs.shape)

torch.Size([4, 19549])


In [48]:
print(enc_outputs.shape)

torch.Size([4, 30, 128])


In [49]:
attn_decoder = AttnDecoderRNN(en_emb_t, hidden_size).cuda()

In [50]:
hidden = attn_decoder.initHidden(batch_size).cuda()

In [51]:
dec_outputs, hidden, attn_weights = attn_decoder(dec_input, hidden, enc_outputs)

In [52]:
print(dec_outputs.shape)
print(hidden.shape)

torch.Size([4, 19549])
torch.Size([2, 4, 128])


In [53]:
topv, topi = dec_outputs.data.topk(1)

In [54]:
ni = [topi[b][0] for b in range(batch_size)]; 
print(ni); print(long_t(ni)); print(long_t(ni))

[14434, 14434, 14434, 14434]
Variable containing:
 14434
 14434
 14434
 14434
[torch.cuda.LongTensor of size 4 (GPU 0)]

Variable containing:
 14434
 14434
 14434
 14434
[torch.cuda.LongTensor of size 4 (GPU 0)]



In [55]:
long_t(ni).unsqueeze(1)

Variable containing:
 14434
 14434
 14434
 14434
[torch.cuda.LongTensor of size 4x1 (GPU 0)]

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.

In [56]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since    # seconds since starting
    es = s / (percent) # seconds expecting finish all iters
    rs = es - s        # seconds remaining to expecting finish
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.


In [57]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Train

Pytorch has limited functionality for training models automatically - you will generally have to write your own training loops. However, Pytorch makes it far easier to customize how this training is done, such as using *teacher forcing*.

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.


In [58]:
def train(input_variable, target_variable, encoder, decoder, 
                     encoder_optimizer, decoder_optimizer, 
                      criterion, max_length=maxlen,  
                      teacher_forcing_ratio=0.5, batch_size=32): 
    decoder_input, encoder_outputs, hidden = encode(input_variable, encoder)
    
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    #input_length = input_variable.size()[1]
    target_length = target_variable.size()[1]
    
    loss = 0
    
    if encoder.n_directions==1:
        decoder_hidden = hidden
    elif encoder.n_directions==2:
        decoder_hidden = decoder.initHidden(batch_size) 
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_variable[:, di])
            decoder_input = target_variable[:, di]  # Teacher forcing
            
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1) # return top (values, indices)
            ni = [topi[b][0] for b in range(batch_size)] #[0][0] # topi shape: (batch_size, 1) 
            
            # *** why pass topi indices as decoder_input? ***
            # ==> Argmax to get the max propability Index
            decoder_input = long_t(ni) #Variable(torch.LongTensor(ni)) # Variable(torch.LongTensor([[ni]]))
            
            #decoder_input = decoder_input.cuda() # if use_cuda else decoder_input
            
            loss += criterion(decoder_output, target_variable[:, di])
            #if ni == EOS_token: 
            #    break
        
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0] / target_length

In [59]:
def req_grad_params(o):
    return (p for p in o.parameters() if p.requires_grad)

In [60]:
def trainEpochs(encoder, decoder, n_epochs, 
               print_every=50, plot_every=25, 
               learning_rate=0.01, 
                batch_size=32):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
        
    encoder_optimizer = optim.RMSprop(req_grad_params(encoder), lr=learning_rate)
    decoder_optimizer = optim.RMSprop(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()
    
    n = len(fr_train)
    n_batch = 0
    n_batch_total = (n % batch_size) * n_epochs 
    for epoch in range(1, n_epochs + 1):
        for batch in range(0, n % batch_size):
            
            fra, eng = get_batch(fr_train, en_train, batch_size)
            input_variable = long_t(fra)
            target_variable = long_t(eng)

            # use_teacher_forcing if running % below teacher_forcing_ratio
            '''
            if (n_batch/n_batch_total) < 0.5:
                teacher_forcing_ratio = 1.0
            elif (n_batch/n_batch_total) < 0.8:
                teacher_forcing_ratio = 0.5
            else: 
                teacher_forcing_ratio = 0.0
            '''
            loss = train(input_variable, target_variable, encoder, decoder, 
                         encoder_optimizer, decoder_optimizer, criterion, 
                        teacher_forcing_ratio=1.0,
                        batch_size=batch_size)

            print_loss_total += loss
            plot_loss_total += loss
            n_batch += 1

            if n_batch % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, n_batch / n_batch_total), 
                                            epoch, (n_batch / n_batch_total * 100),  
                                            print_loss_avg))
            if n_batch % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
    
    showPlot(plot_losses)

## Run

In [61]:
hidden_size = 128
encoder = EncoderRNN(fr_emb_t, hidden_size).cuda()
decoder = AttnDecoderRNN(en_emb_t, hidden_size).cuda()

In [62]:
trainEpochs(encoder, decoder, 500, learning_rate=0.05)

0m 21s (- 90m 51s) (2 0%) 50.2216
0m 42s (- 88m 1s) (4 0%) nan
1m 8s (- 93m 42s) (6 1%) nan
1m 31s (- 94m 8s) (8 1%) nan
1m 52s (- 92m 10s) (10 2%) nan
2m 13s (- 90m 42s) (12 2%) nan
2m 34s (- 89m 37s) (14 2%) nan
2m 55s (- 88m 38s) (16 3%) nan
3m 23s (- 90m 37s) (18 3%) nan
3m 43s (- 89m 33s) (20 4%) nan
4m 4s (- 88m 42s) (22 4%) nan
4m 25s (- 87m 55s) (24 4%) nan
4m 47s (- 87m 15s) (26 5%) nan
5m 12s (- 87m 40s) (28 5%) nan
5m 33s (- 87m 0s) (30 6%) nan
5m 54s (- 86m 24s) (32 6%) nan
6m 15s (- 85m 50s) (34 6%) nan
6m 37s (- 85m 18s) (36 7%) nan
6m 58s (- 84m 49s) (38 7%) nan
7m 20s (- 84m 20s) (40 8%) nan
7m 43s (- 84m 18s) (42 8%) nan
8m 4s (- 83m 44s) (44 8%) nan


KeyboardInterrupt: 